In [36]:
import numpy as np
import random
from typing import Dict, List, Set
import logging


In [21]:
aco_instance = ACOptimizer(ants = 100, evaporation_rate=0.6)
print(aco_instance)

ACO(ants = 100, evaporation_rate = 0.6)


# Step for ACO + Set Cover Code

In [23]:
class SetCoverACO(ACOptimizer):
    
    def __init__(self, universe, subsets, costs):
        """
        
        """
        if len(subsets) != len(costs):
            pass
            

In [4]:
""""Data process"""

class DataReader:


    def __init__(self, file):
        """
        DataReader Initialization
        
        Args:
            file (str): filepath 
        """
        self.file = file
        self.data = self._readTXT()
        self.universe, self.sets = tuple( int(i) for i in self.data[0].split() ) 
        self.sets, self.costs  = self._processTXT(self.data)

    def _readTXT(self):
        """
        Reads filepath.
        """
        with open(self.file) as f:
            data = f.readlines()
        return data

    def _processTXT(self, data):
        """
        Generate sets and costs
        
        Args:
            data (List): List of lines of data read
        
        """
        
        sets = [ [ int(j) for j in i.split()[1: ] ] for i in data[1:] ]
        costs = [ int(i.split()[0])  for i in data[1:] ]
        return  sets, costs


In [7]:
data1 = DataReader(file = "../aco_sc_data/rail582.txt")

In [8]:
len(data1.costs)

55515

# Set Covering class

In [99]:
print(__name__)

__main__


In [133]:
class BaseLogger:
    
    
    def __init__(self, logger_name: str = "Model Logger") -> None:
        
        self.logger_name = logger_name
        self.logger      =  logging.getLogger(self.logger_name)
        self.__logger_settings()
        # TODO: Check if the only handler needed is the StreamHandler
        for handler in self.logger.handlers: self.logger.removeHandler(handler)
        
        
    def __logger_settings(self) -> None:
        
        self.logger.setLevel(logging.INFO)
        logFileHandler = logging.FileHandler("./app.log",mode='w')
        consoleHandler  = logging.StreamHandler()
        logFormatter = logging.Formatter('%(name)s - %(levelname)s - [%(asctime)s] - %(message)s')
        
        consoleHandler.setFormatter(logFormatter)
        logFileHandler.setFormatter(logFormatter)
        
        self.logger.addHandler(logFileHandler)
        self.logger.addHandler(consoleHandler)


In [265]:
test = {
1: 2,
3: 5
}


list(test.keys())

[1, 3]

In [163]:
np.random.choice(range(100), 
                                       size = 1, 
                                       p = None)[0]
        

46

In [268]:
class SetCovering(BaseLogger):
    
    
    def __init__(self, subsets: List, costs: List) -> None:
        """ Set convering initialization
        """
        super().__init__()
        self.subsets = subsets
        self.costs   = costs
        self.universe = self.__identify_unique_items()
        self.total_set_elements = len(self.universe)
        self.total_subsets      = len(subsets)
        self.item_scores        = self.__calculate_item_scores()
        self.set_probabilities  = self.__calculate_set_scores()
    
    def __identify_unique_items(self) -> Set:
        """Find all unique elements of data structure
        
        """
        return { item for instance in self.subsets for item in instance }
    
    def __calculate_item_scores(self):
        item_counts  = dict(
            Counter([item for sublist in self.subsets for item in sublist])
        )
        
        item_values = {}
        for key, value in item_counts.items():
            item_values[key] = 1 / (value / self.total_subsets)
        
        SCORE_MAX = max(item_values.values())
        SCORE_MIN = min(item_values.values())
        
        item_scores = {}
        for key, value in item_values.items():
            item_scores[key] = SetCovering.max_min_normalizer(value, max_val = SCORE_MAX, min_val = SCORE_MIN)
            
        return item_scores

    def __calculate_set_scores(self):
        """Return the average score for each subset"""
        scores = [np.mean([self.item_scores[i] for i in subset])   for subset in self.subsets]
        
        return  self.__calculate_probabilities(scores)
    
    
    def __calculate_probabilities(self, vals):
        total_sum     = sum(vals)
        return [ val / total_sum  for val in vals ]
        
    
    def cover(self, probs = None):
        """ Create a set covering solution
        """
        
        if not probs:
            prob_dist = self.set_probabilities
        
        all_available_subsets = [*range(self.total_subsets)]
        
        
        
        covered = set()
        selected_subsets = []
        cost = 0
        while covered != self.universe:
            
            scores_ = zip(all_available_subsets, self.__calculate_probabilities(prob_dist))
            scores_ = dict(scores_)
            
            subset_idx = self.__select_set(set_list = list( scores_.keys() ),
                                           probs = list( scores_.values() ))
            subset     = set( self.subsets[subset_idx] )
            #print(f"selected set: {subset_idx}")
            
            selected_subsets.append(subset_idx)
            covered |= subset
            
            cost += self.costs[subset_idx]
            
            del scores_[subset_idx]
            
            print(f"total cost: {cost}")
            print(f"total subsets used: {len(selected_subsets)}")
            print(f"elements covered {len(covered)}")
        
        self.logger.info(f">>> Total covering cost: {cost}")
        self.logger.info(f">>> Total subsets selected: {len(selected_subsets)}")
        
        return selected_subsets, cost

        
    def __select_set(self, set_list: List, probs = None):
        """Select a set index, regarding if the set already have appended to a subset_list"""
        subset_idx = np.random.choice( set_list, 
                                       size = 1, 
                                       p = probs)[0]
        return subset_idx
    
    @staticmethod
    def max_min_normalizer(num, max_val, min_val):
        return (num - min_val) / (max_val - min_val)

            

# ACO

In [140]:
"""Ant System Metaherustic"""
class ACOptimizer:
    
    def __init__(self, 
                 set_cover: SetCovering,
                 ants: int = 10,
                 evaporation_rate: float = 0,
                 alpha: float = 1.0,
                 beta:float = 0.0) -> None:
        
        """
        ACO algorithm initialization
        
        Args:
            saet_cover (SetCovering): set cover class with the subsets and costs
            ants (int): Number of ants in the instance
            evaporation_date (float):  pheromone's evaporation speed for each iteration
            
        """
        self.set_cover = set_cover
        self.ants = ants
        self.evaporation_rate = evaporation_rate
        self.heuristic_alpha = alpha
        self.heuristic_beta = beta
        self.set_pheromones = [ 0 for subset in self.set_cover.subsets]
        self.initialized = False
        
    def __str__(self):
        return "ACO(ants = {ants}, evaporation_rate = {evaporation_rate})".format(ants = self.ants, evaporation_rate = self.evaporation_rate)
    
    
    def __calc_obj_function(self):
        pass
    
    def __calc_pheromone(self):
        
        """Calculates pheromone level by each set based on the length of the set
        
        """
        
        if not self.initialized:
            return self.pheromones
        else:
            for i, subset in enumerate(self.set_cover.subsets):
                self.pheromones[i] = len(subset) / self.set_cover.total_subsets
        
        return self.pheromones
            
    
    def __initialize(self):
        
        sets_idx, costs = self.set_cover.random_cover()  
        return sets_idx, costs
    
    def _update_probabilities(self):
        pass
    
    def _select_partial_solution(self):
        pass
    
    def _evaporate(self):
        pass
    
    
    def execute(self, iterations=100, mode='min', early_stopping_count=20, verbose=True) :
        
        
        # 1. Inicialization
        # Calculate pheromones, 
        for i in range(self.ants):
            self.__initialize()
        
        # 2. Ant solutions
        # 3. Local Search
        # 4. Global update pheromones


## Set Covering Pipeline

In [115]:
set_cover_data = DataReader(file = "../aco_sc_data/rail582.txt")

In [116]:
set_cover_data.sets[:10]

[[4, 1, 2, 285, 29],
 [4, 1, 2, 285, 120],
 [4, 1, 2, 285, 132],
 [4, 1, 2, 285, 173],
 [6, 1, 2, 285, 301, 302, 303],
 [4, 1, 2, 302, 297],
 [6, 1, 2, 302, 303, 274, 260],
 [6, 1, 2, 302, 303, 274, 502],
 [8, 1, 2, 3, 87, 575, 559, 576, 560],
 [8, 1, 2, 3, 87, 575, 559, 576, 561]]

In [117]:
set_cover_data.costs[:10]

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [219]:
set_cover_instance.item_scores

AttributeError: 'SetCovering' object has no attribute 'item_scores'

In [269]:
set_cover_instance = SetCovering(subsets = set_cover_data.sets, costs  = set_cover_data.costs)
setssss, costss = set_cover_instance.cover()
#set_cover_instance.set_probabilities

total cost: 2
total subsets used: 1
elements covered 7
total cost: 4
total subsets used: 2
elements covered 13
total cost: 6
total subsets used: 3
elements covered 18
total cost: 7
total subsets used: 4
elements covered 22
total cost: 9
total subsets used: 5
elements covered 28
total cost: 11
total subsets used: 6
elements covered 37
total cost: 12
total subsets used: 7
elements covered 39
total cost: 14
total subsets used: 8
elements covered 47
total cost: 16
total subsets used: 9
elements covered 54
total cost: 18
total subsets used: 10
elements covered 57
total cost: 20
total subsets used: 11
elements covered 64
total cost: 21
total subsets used: 12
elements covered 66
total cost: 23
total subsets used: 13
elements covered 69
total cost: 25
total subsets used: 14
elements covered 75
total cost: 27
total subsets used: 15
elements covered 78
total cost: 28
total subsets used: 16
elements covered 82
total cost: 30
total subsets used: 17
elements covered 85
total cost: 32
total subsets 

Model Logger - INFO - [2021-11-28 17:51:34,221] - >>> Total covering cost: 3374
Model Logger - INFO - [2021-11-28 17:51:34,221] - >>> Total covering cost: 3374
Model Logger - INFO - [2021-11-28 17:51:34,223] - >>> Total subsets selected: 1800
Model Logger - INFO - [2021-11-28 17:51:34,223] - >>> Total subsets selected: 1800


total cost: 3369
total subsets used: 1797
elements covered 581
total cost: 3370
total subsets used: 1798
elements covered 581
total cost: 3372
total subsets used: 1799
elements covered 581
total cost: 3374
total subsets used: 1800
elements covered 582


In [142]:
aco_optimizer = ACOptimizer()

# Results reporting and analytics

In [186]:
from collections import Counter

In [190]:
len(set_cover_data.sets)

55515

In [189]:
b_set = set(tuple(x) for x in set_cover_data.sets)
b = [ list(x) for x in b_set ]
len(b)

55388

In [212]:
test = [4,1,2,502]
[item_final_scores[i] for i in test]

[0.00016770289153780304,
 0.0016527398499460453,
 0.0007210996752265691,
 0.0008815896278277181]

{4: 0.00016770289153780304,
 1: 0.0016527398499460453,
 2: 0.0007210996752265691,
 285: 0.0010342255046072223,
 29: 0.0025572704671827087,
 120: 0.004217660710779753,
 132: 0.002571195202695724,
 173: 0.002799272767133048,
 6: 0.0,
 301: 0.0012276757996821138,
 302: 0.0013030085468822416,
 303: 0.0015230585615243295,
 297: 0.0021932380959138733,
 274: 0.0015179707809310123,
 260: 0.002289737782751793,
 502: 0.0008815896278277181,
 8: 1.1074310651895515e-05,
 3: 0.0010268990945892635,
 87: 0.003816835939832797,
 575: 0.00013978182859855116,
 559: 8.2419118261334e-05,
 576: 4.9589574163676546e-05,
 560: 9.129030014066933e-05,
 561: 9.068780813733374e-05,
 577: 0.00011055735465740539,
 10: 0.000100150450777218,
 579: 0.0001382772647003808,
 563: 8.758208480863199e-05,
 580: 7.73901248073338e-05,
 564: 3.762400153342867e-05,
 562: 0.00019161426963063908,
 61: 0.0007966404940248394,
 578: 0.00014505078004050235,
 7: 3.312293095939981e-05,
 18: 0.004109907608129786,
 131: 0.00473354595784189

In [185]:
set_cover_data.sets

for element in set_cover_data.sets:
    for item in element:

[[4, 1, 2, 285, 29],
 [4, 1, 2, 285, 120],
 [4, 1, 2, 285, 132],
 [4, 1, 2, 285, 173],
 [6, 1, 2, 285, 301, 302, 303],
 [4, 1, 2, 302, 297],
 [6, 1, 2, 302, 303, 274, 260],
 [6, 1, 2, 302, 303, 274, 502],
 [8, 1, 2, 3, 87, 575, 559, 576, 560],
 [8, 1, 2, 3, 87, 575, 559, 576, 561],
 [8, 1, 2, 3, 87, 575, 559, 577, 561],
 [6, 1, 2, 3, 87, 575, 559],
 [6, 1, 2, 3, 87, 575, 560],
 [6, 1, 2, 3, 87, 575, 561],
 [4, 1, 2, 3, 87],
 [10, 1, 2, 3, 120, 577, 561, 579, 563, 580, 564],
 [8, 1, 2, 3, 120, 577, 561, 579, 563],
 [8, 1, 2, 3, 120, 577, 561, 579, 564],
 [8, 1, 2, 3, 120, 577, 561, 580, 564],
 [6, 1, 2, 3, 120, 577, 561],
 [8, 1, 2, 3, 120, 577, 562, 580, 564],
 [6, 1, 2, 3, 120, 577, 562],
 [8, 1, 2, 3, 120, 577, 563, 580, 564],
 [6, 1, 2, 3, 120, 577, 563],
 [6, 1, 2, 3, 120, 577, 564],
 [6, 1, 2, 3, 132, 61, 260],
 [6, 1, 2, 3, 132, 61, 502],
 [8, 1, 2, 3, 132, 576, 560, 578, 562],
 [8, 1, 2, 3, 132, 576, 560, 578, 563],
 [8, 1, 2, 3, 132, 576, 560, 579, 563],
 [6, 1, 2, 3, 132, 576,